In [1]:
import gseapy as gp
import pandas as pd
import numpy as np
import scipy.stats as stats
import glob
from biomart import BiomartServer
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import gseapy as gp

from sklearn import decomposition
from sklearn import preprocessing

from scripts import aesthetics

aesthetics.activate_paper_rcParams()

pd.set_option("display.max_columns", 1000)

# RNA SV Impact

In `neuroblastoma-cwas`, we've discovered that counterintuitively, the SVs we've discovered lead to _increases_ in expression. This is paradoxical from the impact that we expect from the germline context.

I'd like to try examining the impact of _ALL_ SVs on expression. This is going to take a little bit, but it should give us a sense for how some of these SVs are operating

# Read in data

In [2]:
# define SVs and dosages
sv_path = "gs://vanallen-pedsv-analysis/beds/PedSV.v2.5.2.full_cohort.analysis_samples.sites.bed.gz"
dosages_path = "gs://vanallen-pedsv-analysis/beds/PedSV.v2.5.2.full_cohort.analysis_samples.allele_dosages.bed.gz"

# define metadata
metadata_path = "gs://vanallen-pedsv-analysis/sample_info/PedSV.v2.5.2.cohort_metadata.w_control_assignments.tsv.gz"

In [3]:
# list of genes that are annotated - drop ensembl IDs
gene_ref = pd.read_csv("ref/gencode_hg38_protein_coding_genes_for_annotation_7_31_23.txt")
gene_ref = gene_ref[~gene_ref['value'].str.startswith('ENSG00')]
gene_ref = gene_ref['value'].tolist()

Load metadata and SVs

In [4]:
metadata = pd.read_csv(
    metadata_path,
    sep="\t",
)

# add a sex label to metadata
metadata["sex"] = (metadata["chrX_CopyNumber"].round() < 2).astype(int)

Now we load the TPMs to establish what samples are in our dataset.

In [5]:
# generated by a now old notebook - see `old/pgsvs-neuroblastoma-cwas-rna`
merged_tpms = pd.read_csv('data/merged_neuroblastoma_tpms.tsv', sep = '\t')
merged_counts = pd.read_csv('data/merged_neuroblastoma_counts.tsv', sep = '\t')

In [6]:
samples = list(merged_tpms.columns[3:])
len(samples)

209

In [7]:
included_samples = metadata[(metadata['entity:sample_id'].isin(samples)) &
                            (metadata['neuroblastoma_case'] == True)]['entity:sample_id'].tolist()
len(included_samples)

89

In [8]:
merged_tpms = merged_tpms[['gene_id', 'ENSEMBL_ID', 'gene_symbol'] + included_samples]
merged_counts = merged_counts[['gene_id', 'ENSEMBL_ID', 'gene_symbol'] + included_samples]

These are the samples that are in our dosage matrix (others were removed upstream for poor QC). Now we load the SVs. We only keep SVs and dosages that are in our TPM matrix.

In [9]:
###############
### Dosages ###
###############
dosage_cols = pd.read_csv(
    dosages_path,
    sep="\t",
    index_col=False,
    nrows = 0
)

usecols = ['#chr', 'start', 'end', 'ID'] + [s for s in samples if s in dosage_cols.columns]

# load in the dosage data for the samples in TPMs
dosages = pd.read_csv(
    dosages_path,
    sep="\t",
    index_col=False,
    usecols = usecols
)

In [10]:
# identify SVs that have non-zero counts in the dosage matrix 
cohort_dosages = dosages.iloc[:, 4:]

# SVs that are poorly genotyped in more than 20% of samples will be excluded
nan_svs = np.isnan(cohort_dosages).mean(axis = 1) > 0.20

# SVs that have no counts will be excluded
nocount_svs = (cohort_dosages.fillna(0) != 0).sum(axis = 1) == 0

kept_svs = ~(nan_svs | nocount_svs)

In [11]:
cohort_dosages = dosages[kept_svs]

svs_to_analyze = cohort_dosages['ID'].tolist()

In [12]:
###############
##### SVs #####
###############
svs = pd.read_csv(
    sv_path,
    sep="\t",
)

/Users/jettcrowdis/opt/anaconda3/envs/pediatric-germline-svs-3.7.13/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (10,34,35,37,39,41,45,46,47,48,72,73,87,88,106,107,121,122,136,137,155,156,170,171,185,186,204,205,219,220,234,235,253,254,268,269,283,284,302,303,366,367,381,382,400,401,415,416,430,431,449,450,464,465,479,480,498,499,513,514,528,529,547,548,562,563,577,578,596,597,645,646,647,648,649,650,690,691,692,693,694,695,713,714,728,729,743,744,758,759,773,774,788,789,803,804,818,819,833,834,848,849,863,864,878,879,893,894,908,909,923,924,931,937,943) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
svs = svs[svs['name'].isin(svs_to_analyze)].reset_index(drop = True)

In [66]:
svs.to_csv('data/svs-for-sv-rna-analysis.csv', index=False)

We also do some filtering on the TPM matrix.

In [14]:
# keep genes that have at least 10 counts in at least 5 samples (following DESEQ's recommendations)
expressed = (merged_counts[included_samples] >= 10).sum(axis = 1) >= 5

# and fewer than 20% of samples can have zero expression
min_greater_than_zero = (merged_tpms[included_samples] == 0).mean(axis = 1) <= 0.20

# and we remove genes with duplicate gene symbols, as we won't be able to tell (without annoying difficulty)
# which gene is affected by an SV
multigene = merged_tpms['gene_symbol'].value_counts()
multigene = multigene[multigene > 1].index
not_multi = ~(merged_tpms['gene_symbol'].isin(multigene))

kept_rows = expressed & min_greater_than_zero & not_multi

kept_genes = merged_counts[kept_rows]['gene_symbol'].tolist()

In [15]:
len(kept_genes)

23094

And only keep genes in our gene reference, obviously

In [16]:
kept_genes = sorted(set(kept_genes) & set(gene_ref))

In [17]:
len(kept_genes)

14823

With that, let's get into it!

In [18]:
merged_tpms = merged_tpms[merged_tpms['gene_symbol'].isin(kept_genes)]
merged_counts = merged_counts[merged_counts['gene_symbol'].isin(kept_genes)]

# Think briefly about what information we want to carry forward

We want to know how SVs affect the expression of genes around them. Most of our SVs will be singleton or very rare. We need to think about how to systematically examine gene expressions, keeping the info that we want. We have a bit of an expanding problem, in that:

1. A given SV can affect multiple genes. I expect the most interesting effects to be on single gene SVs, but we should look at all of them.
2. Multiple samples can have an SV. Common SVs can affect multiple SVs and could probably be handled by a MWU, but singleton or rare SVs cannot.
3. SVs can have different dosages. We'll be tracking CNVs here, which can vary dramatically in terms of their dosages.

It's very difficult to combine all this information into a single dataframe. In addition, we occasionally want different information. For common SVs, we can directy compare expression, but for rare and singleton SVs, a rank-based approach is likely to work better.

I'm just going to make this up as I go along.

# Identify SVs that affect genes

Here, we'll identify SVs that nominally affect genes. At the end of this process, I want to end up with an SV x gene flat dataframe, where each row contains information about the SV and its relationship to the gene. This will carry forward a lot of redundant information about the SV, but that's ok.

In [22]:
coding_cols = ['PREDICTED_COPY_GAIN', 'PREDICTED_INTRAGENIC_EXON_DUP', 'PREDICTED_LOF', 'PREDICTED_PARTIAL_EXON_DUP']
noncoding_cols = ['PREDICTED_NEAREST_TSS', 'PREDICTED_INTRONIC', 'PREDICTED_PROMOTER', 'PREDICTED_UTR']

In [20]:
svs_that_affect_genes = (~pd.isnull(svs[coding_cols + noncoding_cols])).sum(axis = 1) > 0

In [21]:
gene_svs = svs[svs_that_affect_genes]
gene_svs.shape[0], gene_svs.shape[0] / svs.shape[0] 

(31397, 0.9932616260677001)

Somewhat surprisingly, apparently 99% of these SVs are associated with a gene. I suppose these are likely all noncoding effects for distant genes.

In [22]:
coding_svs = (~pd.isnull(svs[coding_cols])).sum(axis = 1) > 0
coding_svs.sum()

594

That's better. Alright, let's cobble this together.

In [23]:
sv_gene_df = []
for sv_name, row in gene_svs[['name'] + coding_cols + noncoding_cols].set_index('name').iterrows():
    row = row[~pd.isnull(row)]
    
    for genic_rel, gene_list in row.iteritems():
        genic_cat = 'coding' if genic_rel in coding_cols else 'non-coding'
        
        for gene in gene_list.split(','):
            sv_gene_df.append([sv_name, genic_cat, genic_rel, gene])
        
sv_gene_df = pd.DataFrame(sv_gene_df, columns = ['name', 'sv_effect', 'genic_relationship', 'gene'])

for sv_effect in ['coding', 'non-coding']:
    counts = pd.DataFrame(sv_gene_df.query(f'sv_effect == "{sv_effect}"').groupby('name').size().astype(int), 
                          columns = [f'sv_{sv_effect}_counts']).reset_index()
    sv_gene_df = sv_gene_df.merge(counts, how = 'left')
    sv_gene_df[f'sv_{sv_effect}_counts'] = sv_gene_df[f'sv_{sv_effect}_counts'].fillna(0)

# add some info about the SVs themselves
sv_gene_df = sv_gene_df.merge(gene_svs[['#chrom', 'start', 'end', 'name', 'svtype']], on = ['name'], how = 'left')

In [24]:
sv_gene_df.head(2)

,name,sv_effect,genic_relationship,gene,sv_coding_counts,sv_non-coding_counts,#chrom,start,end,svtype
0,PedSV.2.5.2_CNV_chr1_1,non-coding,PREDICTED_NEAREST_TSS,OR4F5,0.0,1.0,chr1,12000,30001,CNV
1,PedSV.2.5.2_DUP_chr1_1,non-coding,PREDICTED_NEAREST_TSS,OR4F5,0.0,1.0,chr1,12000,40001,DUP


In [25]:
(sv_gene_df.groupby(['name']).size() == 1).sum() / gene_svs.shape[0]

0.9909863999745199

So we can see that the majority of SVs actually only affect one gene (and this remains true for coding genes too).

I'd like to add some info about AF in our cohort subset here. Calculating these are _EXTREMELY_ annoying, since we have mixed CNVs and short SVs.

In [26]:
sv_allele_fractions = []

for index, row in cohort_dosages.iterrows():
    
    dosages = row.iloc[4:].dropna()
    
    if '_CNV' in row['ID']:
        average_cn = np.mean(dosages)
        af = np.nan
        num_samples_affected = (dosages != 0).sum()
        
    else:
        allele_counts = dosages.value_counts().reindex([0, 1, 2]).fillna(0)
        alt = (allele_counts * np.array([0, 1, 2])).sum()
        ref = len(dosages) * 2
        
        af = alt/ref
        average_cn = np.nan
        num_samples_affected = (dosages > 0).sum()
        
    num_genotyped = len(dosages)
        
    sv_allele_fractions.append([row['ID'], num_samples_affected, num_genotyped, af, average_cn])
    
sv_allele_fractions = pd.DataFrame(sv_allele_fractions, columns = ['name', 'num_samples_nonzero', 'num_samples_genotyped', 'af', 'average_cn'])
        

In [27]:
sv_allele_fractions.head(2)

,name,num_samples_nonzero,num_samples_genotyped,af,average_cn
0,PedSV.2.5.2_CNV_chr1_1,46,89,NaN,0.741573
1,PedSV.2.5.2_DUP_chr1_1,2,81,0.012346,NaN


In [28]:
sv_gene_df = sv_gene_df.merge(sv_allele_fractions, on = 'name', how = 'left')

In [29]:
sv_gene_df = sv_gene_df[['name', '#chrom', 'start', 'end', 'svtype', 'sv_coding_counts', 'sv_non-coding_counts', 
                         'num_samples_nonzero', 'num_samples_genotyped', 'af', 'average_cn', 'sv_effect', 'genic_relationship', 'gene']]

In [30]:
sv_gene_df.head(2)

,name,#chrom,start,end,svtype,sv_coding_counts,sv_non-coding_counts,num_samples_nonzero,num_samples_genotyped,af,average_cn,sv_effect,genic_relationship,gene
0,PedSV.2.5.2_CNV_chr1_1,chr1,12000,30001,CNV,0.0,1.0,46,89,NaN,0.741573,non-coding,PREDICTED_NEAREST_TSS,OR4F5
1,PedSV.2.5.2_DUP_chr1_1,chr1,12000,40001,DUP,0.0,1.0,2,81,0.012346,NaN,non-coding,PREDICTED_NEAREST_TSS,OR4F5


In [31]:
sv_gene_df.shape

(31885, 14)

# Time to look at the RNA

I want to do this fully systematically, and then afterwards we can go back and reassess. We have a few different issues that we need to handle:

1. Different numbers of samples affected. Some SVs affect many samples, and some affect very few.
2. SV dosages. While rare SVs will usually only have `0` or `1` as dosages, others will have `1/2`. 
3. `CNV`s have way crazier dosages. They should be modelled more holistically.
4. Modelling TPMs/counts as an outcome is really annoying (requiring something like DESeq optimally), and non-parametric approaches like the MWU cannot handle multiple covariates.

This is a ton to keep track of, as we want to handle these scenarios differently. Here's what we'll do. For each `SV` and `gene` pair, we'll generate the following:

1. The average rank of affected samples (non-zero allele). This will be incorrect for CNVs that can have a range of copy numbers that includes negative ones.
    * We also include the expression of affected vs. not samples
2. MWU test between affected (non-zero allele) and not. This will be incorrect for SVs that have few samples and for SVs that have more dosages.
3. An ordinal logistic regression model, incorporating dosage and using the ranks of gene expression as the outcome. This will be broadly incorrect because it does not appropriately model counts.
4. The average expression of the gene

In [32]:
from statsmodels.miscmodels.ordinal_model import OrderedModel

In [33]:
# we get to begin by subsetting our gene df to genes that are in our TPM matrix
sv_gene_df = sv_gene_df[sv_gene_df['gene'].isin(merged_tpms['gene_symbol'])]

# format for analysis
analysis_dosages = cohort_dosages.set_index('ID').iloc[:, 3:]
analysis_tpms = merged_tpms.set_index('gene_symbol').iloc[:, 2:]

In [64]:
# do some exporting for easy reading in later
merged_tpms.to_csv('data/tpms-for-sv-rna-analysis.csv', index=False)
cohort_dosages.to_csv('data/dosages-for-sv-rna-analysis.csv', index=False)

In [92]:
results = []
for i, (index, row) in enumerate(sv_gene_df.iterrows()):
    if i % 500 == 0:
        print(i, end = ', ')

    gene = row['gene']
    sv = row['name']
    gr = row['genic_relationship']

    storage_row = [sv, gene, gr]

    # get the dosages
    sv_dosages = analysis_dosages.loc[sv].dropna()

    # get the expression
    gene_expression = analysis_tpms.loc[gene, sv_dosages.index]
    mean_expression = gene_expression.mean()

    storage_row.append(mean_expression)

    ###########################
    ### RANK-BASED APPROACH ###
    ###########################
    affected_samples = sv_dosages[sv_dosages > 0].index
    unaffected_samples = sv_dosages[sv_dosages <= 0].index

    # rank the expression
    expression_ranks = (gene_expression.rank(ascending = False) - 1)
    norm_expression_ranks =  expression_ranks / (len(gene_expression) - 1)

    avg_affected_rank = norm_expression_ranks.loc[affected_samples].mean()

    # store this data
    storage_row += [len(affected_samples), avg_affected_rank]

    ################
    ### MWU TEST ###
    ################
    gene_exp_affected = gene_expression.loc[affected_samples]
    gene_exp_unaffected = gene_expression.loc[unaffected_samples]
    try:
        p = stats.mannwhitneyu(gene_exp_affected, gene_exp_unaffected)[1]
    except:
        p = np.nan

    storage_row += [gene_exp_affected.mean(), gene_exp_unaffected.mean(), p]

    ###################################
    ### LOGISTIC ORDINAL REGRESSION ###
    ###################################
    data = pd.DataFrame([expression_ranks, sv_dosages], index = ['rank', 'dose']).T

    # have to convert rank to an ordered variable
    data['rank'] = pd.Categorical(data['rank'], categories=sorted(set(data['rank']))[::-1], ordered=True)

    try:
        mod_log = OrderedModel(data['rank'],
                       data[['dose']],
                       distr='logit')

        res_log = mod_log.fit(method='bfgs', disp=False)

        p = res_log.pvalues.loc['dose']
        coef = res_log.params.loc['dose']

    except:
        p, coef = np.nan, np.nan

    storage_row += [p, coef]

    results.append(storage_row)

0, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500, 6000, 6500, 7000, 7500, 8000, 8500, 9000, 9500, 10000, 10500, 11000, 11500, 12000, 12500, 13000, 13500, 14000, 14500, 15000, 15500, 16000, 16500, 17000, 17500, 18000, 18500, 19000, 19500, 20000, 20500, 21000, 21500, 22000, 22500, 23000, 23500, 24000, 24500, 25000, 

In [96]:
columns = ['name', 'gene', 'genic_relationship', 'mean_exp', 'num_greater_0_dosage', 'mean_greater_0_dosage_rank', 'mean_greater_0_exp', 'mean_leq_0_exp', 
           'mwu_p', 'ordinal_p', 'ordinal_coef']

In [97]:
results = pd.DataFrame(results, columns = columns)

And combine it with some of the SV characteristics

In [132]:
results.head(2)

,name,gene,mean_exp,num_greater_0_dosage,mean_greater_0_dosage_rank,mean_greater_0_exp,mean_leq_0_exp,mwu_p,ordinal_p,ordinal_coef,genic_relationship
0,PedSV.2.5.2_CNV_chr1_6,OR4F29,0.266629,15,0.507955,0.247333,0.270541,0.912668,0.582179,-0.188116,PREDICTED_NEAREST_TSS
1,PedSV.2.5.2_CNV_chr1_7,OR4F29,0.266629,62,0.510264,0.255484,0.292222,0.620135,0.461080,-0.181469,PREDICTED_NEAREST_TSS


In [130]:
sv_gene_df = sv_gene_df.merge(results, on = ['name', 'gene', 'genic_relationship'], how = 'left')

Finally, I do want to swap the rank around, such that `0 = low | 1 = high`. 

In [70]:
sv_gene_df['mean_greater_0_dosage_rank'] = 1 - sv_gene_df['mean_greater_0_dosage_rank']

In [71]:
sv_gene_df.to_csv('data/sv-expression-results/sv-gene-rna-results.csv', index=False)